# Pattern matching

* https://opencv24-python-tutorials.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_template_matching/py_template_matching.html
* https://docs.opencv.org/3.4.15/df/dfb/group__imgproc__object.html#ga586ebfb0a7fb604b35a23d85391329be

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

## Read the input images

In [ ]:
im_input = cv2.imread("input/pacman.png")
plt.imshow(im_input);

In [ ]:
im_pattern = cv2.imread("input/input.png")
plt.imshow(im_pattern);

## Similarity measure
* https://docs.opencv.org/3.4.15/df/dfb/group__imgproc__object.html#ga3a7850640f1fe1f58fe91a2d7583695d
* https://stackoverflow.com/questions/55469431/what-does-the-tm-ccorr-and-tm-ccoeff-in-opencv-mean


In [ ]:
def sim_ccorr(m1, m2):  # https://numpy.org/doc/stable/reference/generated/numpy.multiply.html
    return np.sum(m1*m2)

## Procedure

### Raw implementation

In [ ]:
im = cv2.cvtColor(im_input, cv2.COLOR_BGR2GRAY).astype(np.uint32)
im_template = cv2.cvtColor(im_pattern, cv2.COLOR_BGR2GRAY).astype(np.uint32)

In [ ]:
h, w = im.shape
template_h, template_w = im_template.shape

In [ ]:
%%time
sim_map = np.zeros((h - template_h, w-template_w)) 
for y in range(h - template_h):
    for x in range (w-template_w):
        sim_map[y, x] = sim_ccorr(im[y:y+template_h, x:x+template_w], im_template)

In [ ]:
plt.imshow(sim_map, cmap = 'gray');

In [ ]:
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(sim_map)

top_left = max_loc
bottom_right = (top_left[0] + template_w, top_left[1] + template_h)

im_show = im_input.copy()
cv2.rectangle(im_show, top_left, bottom_right, (0, 255, 0), 5)
plt.imshow(im_show);

In [ ]:
threshold = 0.85* np.max(sim_map)

In [ ]:
im_show = im_input.copy()
y, x = np.where(sim_map >= threshold)
for pt in zip(x, y):
    cv2.rectangle(im_show, pt, (pt[0] + template_w, pt[1] + template_h), (0,255,0), 5)

In [ ]:
plt.imshow(im_show);

### OpenCV implementation

In [ ]:
# Required image codification for the algorithm
im = im.astype(np.uint8)
im_template = im_template.astype(np.uint8)

In [ ]:
%%time
res = cv2.matchTemplate(im,im_template,cv2.TM_CCORR) 

In [ ]:
res = cv2.matchTemplate(im,im_template,cv2.TM_CCORR)

im_show = im_input.copy()
threshold = 0.85* np.max(res)
loc = np.where( res >= threshold)
for pt in zip(*loc[::-1]):
    cv2.rectangle(im_show, pt, (pt[0] + template_w, pt[1] + template_h), (0,0,255), 2)

plt.subplot(121),plt.imshow(res, cmap = 'gray')
plt.title('Matching Result'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(im_show,cmap = 'gray')
plt.title('Detected Point'), plt.xticks([]), plt.yticks([]);

## Tasks
* Compare execution time between the two implementations.
* Why there are some pacmans better detected than others?
* Change the size of the pattern image (cv2.resize). How do the results change?
* Make a slight rotation of the pattern image (cv2.rotate). How do the results change?
* What could be the utility of the rotation and the resize of the template image?

In [ ]:
plt.imshow(im_template)

In [ ]:
image = cv2.rotate(im_template, cv2.cv2.ROTATE_90_CLOCKWISE)
plt.imshow(image)

To rotate a specific number of degrees you can use the function: cv2.wrapAffine

https://www.pyimagesearch.com/2017/01/02/rotate-images-correctly-with-opencv-and-python/

In [ ]:
im_template.shape

In [ ]:
new_shape = (100, 100)
image = cv2.resize(im_template, new_shape, interpolation= cv2.INTER_LINEAR)
plt.imshow(image)